In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

DATA_PATH = 'transcrittomic'

c:\Users\giuli\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
adata = sc.read(DATA_PATH)
adata

AnnData object with n_obs × n_vars = 250920 × 0
    obs: 'dataset', 'disease', 'cell_type', 'sample', 'patient_id', 'time', 'cell_types_labels', 'patient', 'MS/HC'
    var: 'gene_ids-GSE138266', 'gene_symbols-GSE138266', 'gene_ids-GSE194078', 'feature_types-GSE194078', 'gene_ids-GSE239626', 'feature_types-GSE239626'

In [3]:
adata.obs.head()

,dataset,disease,cell_type,sample,patient_id,time,cell_types_labels,patient,MS/HC
AAACCCAAGACTGTTC-1-GSE239626,GSE239626,MS,PBMC,GSM7669046,N1,J0,T cells,NaN,MS
AAACCCAAGGATCATA-1-GSE239626,GSE239626,MS,PBMC,GSM7669046,N1,J0,T cells,NaN,MS
AAACCCAGTTATTCTC-1-GSE239626,GSE239626,MS,PBMC,GSM7669046,N1,J0,B cells,NaN,MS
AAACCCATCATGAGGG-1-GSE239626,GSE239626,MS,PBMC,GSM7669046,N1,J0,T cells,NaN,MS
AAACGAAAGCCAGTAG-1-GSE239626,GSE239626,MS,PBMC,GSM7669046,N1,J0,B cells,NaN,MS


In [4]:
adata.obs.describe()

,dataset,disease,cell_type,sample,patient_id,time,cell_types_labels,patient,MS/HC
count,250920,250920,250920,140355,169641,72317,250920,178603,250920
unique,3,6,2,39,16,2,19,23,2
top,GSE194078,MS,PBMC,GSM4104138,KSH,M3,T cells,GSM5827380,MS
freq,110565,151730,177189,7819,22578,37203,193689,22578,151730


In [5]:
adata.X

<250920x0 sparse matrix of type '<class 'numpy.float32'>'
	with 0 stored elements in Compressed Sparse Row format>

In [6]:
adata.var.describe()

,gene_ids-GSE138266,gene_symbols-GSE138266,gene_ids-GSE194078,feature_types-GSE194078,gene_ids-GSE239626,feature_types-GSE239626
count,0,0,0,0,0,0
unique,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN
